In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - make_iso.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Do some processing on the raw PARSEC v1.2 isochrones to make an isochrone
grid which can be used with the mass fitting program. Specifically, add 
weights which represent the fractional number density of each isochrone point.
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, copy
from matplotlib import pyplot as plt
import scipy.integrate

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import util as putil

### Scale parameters
ro = 8.
vo = 220
zo = 0.0208 # Bennett+ 2019

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Preliminaries

In [ ]:
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR']
data_dir_base,apogee_dr,apogee_results_vers,gaia_dr \
    = putil.parse_config_dict(cdict,keywords)
data_dir = data_dir_base+'gaia_apogee/'
gaia_apogee_dir = 'apogee_'+apogee_dr+'_'+apogee_results_vers+'_gaia_'+gaia_dr+'/'

### Load data

In [ ]:
# Path for isochrone, I keep it where data for isodist lives
iso_dir = os.environ['ISODIST_DATA']+'/parsec1.2/'
# Two isochrones, one for low metallicity and one for higher metallicity.
# Done separately because of CMD 3.6 limitations, now joined.
iso_names = ['2mass-spitzer-wise-0.0001-z-0.0030-1e10-age-1.4e10.dat',
             '2mass-spitzer-wise-0.0031-z-0.0060-1e10-age-1.4e10.dat']
iso_1 = np.genfromtxt(iso_dir+iso_names[0], names=True, skip_header=13, 
                      comments='#')
iso_2 = np.genfromtxt(iso_dir+iso_names[1], names=True, skip_header=13, 
                      comments='#')
iso = np.concatenate((iso_1,iso_2))

### Processing
First calculate the IMF-based weights and include them in the isochrone. The 
IMF for each isochrone element is the delta of iso['int_IMF'], calculated 
individually for each isochrone of a unique Z and Age. For the first mass point 
I integrate a normalized (1 M solar total) Chabrier (2001) IMF from 0.08 to the 
first initial mass point.

Additionally remove any points with luminosity -9.999, which are WDs that can 
significantly warp the inferred mass fraction

In [ ]:
def chabrier01(m,A=1.):
    sigma = 0.627
    m0 = 0.1
    logm_imf = 0.141*np.exp(-np.square(np.log10(m)-np.log10(m0))/(2.*sigma**2.))
    return A*logm_imf/np.log(10)/m

chabrier01_imf_mass = lambda mass: chabrier01(mass)*mass

In [ ]:
imf_norm = scipy.integrate.quad(chabrier01_imf_mass,0.,100.)[0]

In [ ]:
unique_zini = np.unique(iso['Zini'])
unique_age = np.unique(iso['logAge'])

w_imf = np.zeros(len(iso),dtype=[('weights_imf','f8')])

for i in range(len(unique_zini)):
    for j in range(len(unique_age)):
        iso_mask = np.where((iso['Zini']==unique_zini[i]) &\
                            (iso['logAge']==unique_age[j]))[0]
        w_imf[iso_mask[1:]] = np.diff(iso[iso_mask]['int_IMF'])
        w_imf[iso_mask[0]] = scipy.integrate.quad(chabrier01, 0.08, 
            iso[iso_mask[0]]['Mini'], args=(1/imf_norm))[0]
        # w_imf[iso_mask[0]] = iso[iso_mask[0]]['int_IMF']

In [ ]:
# mavg = np.zeros((len(unique_zini),len(unique_age)))
# mtot = np.zeros((len(unique_zini),len(unique_age)))

# for i in range(len(unique_zini)):
#     for j in range(len(unique_age)):
#         iso_mask = np.where((iso['Zini']==unique_zini[i]) &\
#                             (iso['logAge']==unique_age[j]) &\
#                             (iso['logL'] > -9)
#                            )[0]
        
#         mavg[i,j] = np.sum(iso['Mini'][iso_mask]*w_imf[iso_mask])/np.sum(w_imf[iso_mask])
#         mtot[i,j] = np.sum(iso['Mini'][iso_mask]*w_imf[iso_mask])
        
#         print('Z='+str(unique_zini[i])+', age='+str(unique_age[j])+\
#               ' total mass: '+str(mtot[i,j])+', average_mass: '+str(mavg[i,j]))

In [ ]:
# Add the weights and remove WD points
iso = np.lib.recfunctions.merge_arrays((iso,w_imf), flatten=True)
iso = iso_merge[iso_merge['logL'] < -9]

In [ ]:
iso_merge.dtype.names

In [ ]:
# Save, store it with all of the other things required to run the density 
# fitting program
np.save(data_dir+gaia_apogee_dir+'iso_grid.npy',iso_merge)